This is a separate notebook because I want to run multiple at the same time.
This is because the API limit our call per 1 minutes so we have to run multiple kernel (using a different API key) to speed up the process

In [8]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import sys 
import numpy as np
#sys.path.append('C:/Users/linhc/AppData/Local/Packages/PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0/LocalCache/local-packages/Python39/site-packages')
# golbal variables
#FlyingCrab account
api_key = 'RGAPI-c2cc0bb8-ee0a-4c11-8210-58efe3db20d5'
watcher = LolWatcher(api_key)
my_region = 'na1'

In [2]:
matchesIdDF = pd.read_excel(r'C:/Users/linhc/OneDrive/Documents/Sproj/DataCollection/AverageMatchesId.xlsx')
matchesId = []
for i in range(0,len(matchesIdDF)):
    matchesId.append(matchesIdDF.values[i][1])
print(len(matchesId))
print(matchesId)

75003
['NA1_4046259518', 'NA1_4061495645', 'NA1_4060586360', 'NA1_4061036860', 'NA1_4022356802', 'NA1_3982005976', 'NA1_4060273914', 'NA1_4059426708', 'NA1_4047303867', 'NA1_4059705573', 'NA1_4061058812', 'NA1_4059592542', 'NA1_4060553143', 'NA1_4055992942', 'NA1_4045195252', 'NA1_4034145402', 'NA1_4061756091', 'NA1_4058333164', 'NA1_4001840686', 'NA1_4053141652', 'NA1_4018542090', 'NA1_4061892704', 'NA1_4061218739', 'NA1_3941591854', 'NA1_3920456043', 'NA1_4042578943', 'NA1_4049372438', 'NA1_4016661731', 'NA1_4058336809', 'NA1_4045637910', 'NA1_3809444445', 'NA1_4056694007', 'NA1_4060821454', 'NA1_4028492931', 'NA1_4013338348', 'NA1_4060737282', 'NA1_4060853521', 'NA1_4022835882', 'NA1_4015140712', 'NA1_4059552810', 'NA1_4061368510', 'NA1_4060787776', 'NA1_4015404184', 'NA1_3987211226', 'NA1_4051015820', 'NA1_4060283466', 'NA1_4022199381', 'NA1_4054844722', 'NA1_3876020493', 'NA1_4034369902', 'NA1_4057332113', 'NA1_4060394343', 'NA1_4038649313', 'NA1_4060592410', 'NA1_4053843633', 'NA

Separate into parts so it will be easier to handle errors

In [3]:
dataSets = np.array_split(matchesId,15)
for i in range(15):
    print(len(dataSets[i]))


5001
5001
5001
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000
5000


## Getting the first 14 mins of each games 

### The Function to gather data and process data

In [4]:
def DataProcessing(matchesIds):
    matchesData=[]
    for matchNo in range(len(matchesIds)):
        try:
            match = watcher.match_v5.by_id('AMERICAS',matchesIds[matchNo])
            matchTime = watcher.match_v5.timeline_by_match('AMERICAS',matchesIds[matchNo])
            #getting the 14th frame for champion info
            frame10 = matchTime['info']['frames'][14]['participantFrames']

            matchesDataRow ={}

            matchesDataRow['blueWins']= 0

            matchesDataRow['blueWardsPlaced'] = 0
            matchesDataRow['blueWardsDestroyed']=0
            matchesDataRow['blueKills']= 0
            matchesDataRow['blueDeaths']= 0
            matchesDataRow['blueAssists']= 0
            matchesDataRow['blueEliteMonsters']= 0
            matchesDataRow['blueDragons']= 0
            matchesDataRow['blueHeraldsBarons']= 0
            matchesDataRow['blueTowersDestroyed']= 0
            matchesDataRow['blueTotalGold']= 0
            matchesDataRow['blueTotalDamageToChampions'] = 0
            matchesDataRow['blueAvgLevel']= 0
            matchesDataRow['blueTotalMinionsKilled']= 0
            matchesDataRow['blueTotalJungleMinionsKilled']= 0
            matchesDataRow['blueGoldDiff']= 0

            matchesDataRow['redWardsPlaced'] = 0
            matchesDataRow['redWardsDestroyed']= 0
            matchesDataRow['redKills']= 0
            matchesDataRow['redDeaths']= 0
            matchesDataRow['redAssists']= 0
            matchesDataRow['redEliteMonsters']= 0
            matchesDataRow['redDragons']= 0
            matchesDataRow['redHeraldsBarons']= 0
            matchesDataRow['redTowersDestroyed']= 0
            matchesDataRow['redTotalGold']= 0
            matchesDataRow['redTotalDamageToChampions'] = 0
            matchesDataRow['redAvgLevel']= 0
            matchesDataRow['redTotalMinionsKilled']= 0
            matchesDataRow['redTotalJungleMinionsKilled']= 0
            matchesDataRow['redGoldDiff']= 0

            #Gather info in the timeline for the first 14 mins
            for i in range(0,15):
                events = matchTime['info']['frames'][i]['events']
                for event in events:
                    #
                    if(event['type']=='WARD_PLACED'):
                        if (event['creatorId'] == 1 or event['creatorId'] == 2 or event['creatorId'] == 3 or event['creatorId'] == 4 or event['creatorId'] == 5):
                            matchesDataRow['blueWardsPlaced'] = matchesDataRow['blueWardsPlaced'] + 1
                        else:
                            matchesDataRow['redWardsPlaced'] = matchesDataRow['redWardsPlaced'] + 1
                    #
                    if(event['type']=='WARD_KILL'):
                        if (event['killerId'] == 1 or event['killerId'] == 2 or event['killerId'] == 3 or event['killerId'] == 4 or event['killerId'] == 5):
                            matchesDataRow['blueWardsDestroyed'] = matchesDataRow['blueWardsDestroyed'] + 1
                        else:
                            matchesDataRow['redWardsDestroyed'] = matchesDataRow['redWardsDestroyed'] + 1
                    #
                    if(event['type']=='CHAMPION_KILL'):
                        if (event['killerId'] == 1 or event['killerId'] == 2 or event['killerId'] == 3 or event['killerId'] == 4 or event['killerId'] == 5):
                            matchesDataRow['blueKills'] = matchesDataRow['blueKills'] + 1
                            matchesDataRow['redDeaths'] =  matchesDataRow['redDeaths']+1
                            try:
                                matchesDataRow['blueAssists'] = matchesDataRow['blueAssists'] + len(event['assistingParticipantIds'])
                            except:
                                matchesDataRow['blueAssists'] = matchesDataRow['blueAssists']
                        else:
                            matchesDataRow['redKills'] = matchesDataRow['redKills'] + 1
                            matchesDataRow['blueDeaths'] = matchesDataRow['blueDeaths']+1
                            try:
                                matchesDataRow['redAssists'] = matchesDataRow['redAssists'] + len(event['assistingParticipantIds'])
                            except:
                                matchesDataRow['redAssists'] = matchesDataRow['redAssists']

                    #
                    if(event['type']=='ELITE_MONSTER_KILL'):
                        if (event['killerId'] == 1 or event['killerId'] == 2 or event['killerId'] == 3 or event['killerId'] == 4 or event['killerId'] == 5):
                            matchesDataRow['blueEliteMonsters'] = matchesDataRow['blueEliteMonsters'] + 1
                            if(event['monsterType'] == 'DRAGON'):
                                matchesDataRow['blueDragons']= matchesDataRow['blueDragons'] + 1
                            else:
                                matchesDataRow['blueHeraldsBarons']= matchesDataRow['blueHeraldsBarons'] + 1
                        else:
                            matchesDataRow['redEliteMonsters'] = matchesDataRow['redEliteMonsters'] + 1
                            if(event['monsterType'] == 'DRAGON'):
                                matchesDataRow['redDragons']= matchesDataRow['redDragons'] + 1
                            else:
                                matchesDataRow['redHeraldsBarons']= matchesDataRow['redHeraldsBarons'] + 1
                    #
                    if(event['type']=='BUILDING_KILL'):
                        if (event['killerId'] == 1 or event['killerId'] == 2 or event['killerId'] == 3 or event['killerId'] == 4 or event['killerId'] == 5):
                            matchesDataRow['blueTowersDestroyed'] = matchesDataRow['blueTowersDestroyed'] + 1
                        else:
                            matchesDataRow['redTowersDestroyed'] = matchesDataRow['redTowersDestroyed'] + 1

            temp = match['info']['teams'][0]
            if temp['teamId'] == 100:
                if  temp['win']== True:
                    matchesDataRow['blueWins']= 1
                elif temp['win']== False:
                    matchesDataRow['blueWins']= 0
            elif(temp['teamId'] == 200):
                if  temp['win']== True:
                    matchesDataRow['blueWins']= 0
                elif temp['win']== False:
                    matchesDataRow['blueWins']= 1

            for i in range(1,6):
                matchesDataRow['blueTotalDamageToChampions']= matchesDataRow['blueTotalDamageToChampions'] + frame10[str(i)]['damageStats']['totalDamageDoneToChampions']
                matchesDataRow['blueTotalGold']= matchesDataRow['blueTotalGold'] + frame10[str(i)]['totalGold']
                matchesDataRow['blueAvgLevel']= matchesDataRow['blueAvgLevel'] + frame10[str(i)]['level']
                matchesDataRow['blueTotalMinionsKilled']= matchesDataRow['blueTotalMinionsKilled'] + frame10[str(i)]['minionsKilled']
                matchesDataRow['blueTotalJungleMinionsKilled']= matchesDataRow['blueTotalJungleMinionsKilled'] + frame10[str(i)]['jungleMinionsKilled']
            matchesDataRow['blueAvgLevel']= matchesDataRow['blueAvgLevel'] / 5

            for i in range(6,11):
                matchesDataRow['redTotalDamageToChampions']= matchesDataRow['redTotalDamageToChampions'] + frame10[str(i)]['damageStats']['totalDamageDoneToChampions']
                matchesDataRow['redTotalGold']= matchesDataRow['redTotalGold'] + frame10[str(i)]['totalGold']
                matchesDataRow['redAvgLevel']= matchesDataRow['redAvgLevel'] +  frame10[str(i)]['level']
                matchesDataRow['redTotalMinionsKilled']= matchesDataRow['redTotalMinionsKilled'] + frame10[str(i)]['minionsKilled']
                matchesDataRow['redTotalJungleMinionsKilled']= matchesDataRow['redTotalJungleMinionsKilled'] + frame10[str(i)]['jungleMinionsKilled']
            matchesDataRow['redAvgLevel']= matchesDataRow['redAvgLevel'] / 5

            matchesDataRow['blueGoldDiff']= matchesDataRow['blueTotalGold'] - matchesDataRow['redTotalGold']
            matchesDataRow['redGoldDiff']= matchesDataRow['redTotalGold'] - matchesDataRow['blueTotalGold']
            matchesData.append(matchesDataRow)
            print(matchNo)
        except:
            print('error')
    return matchesData


In [5]:
dataSet0Result = DataProcessing(dataSets[0])
print(len(dataSet0Result))
print('Done')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
error
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
error
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
error
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
error
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
error
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
27

1846
1847
1848
1849
1850
1851
1852
error
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
error
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
error
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
error
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2

3477
3478
3479
3480
3481
error
3483
3484
3485
3486
3487
3488
3489
3490
3491
3492
3493
3494
3495
3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
error
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
367

In [6]:
dataSet1Result = DataProcessing(dataSets[1])
print(len(dataSet1Result))
print('DONE')

0
1
2
3
4
5
6
error
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
error
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
error
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
error
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
error
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
error
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
error
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272


1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044


3477
3478
3479
3480
3481
3482
3483
3484
3485
3486
3487
3488
3489
3490
3491
3492
3493
3494
3495
3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
error
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
error
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
367

In [7]:
dataSet2Result = DataProcessing(dataSets[2])
print(len(dataSet2Result))
print('DONE')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
error
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
error
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
error
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
error
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
error
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
27

1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
error
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
error
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
error
1923
error
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
error
1941
1942
1943
1944
1945
error
1947
1948
1949
1950
1951
error
1953
1954
1955
1956
1957
1958
1959
1960
1961
error
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
error
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
error
2037
2038
2039
2040
error
2042
2043
2044
2045

3478
error
error
3481
3482
3483
3484
3485
3486
3487
3488
3489
3490
3491
3492
3493
3494
3495
3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
error
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
error
3528
3529
3530
3531
3532
error
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
error
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
error
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
error
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
36

In [9]:
dataSet3Result = DataProcessing(dataSets[3])
print(len(dataSet3Result))
print('DONE')

error
error
error
error
error
error
error
error
error
error
10
error
error
13
error
error
16
17
18
19
error
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
error
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
error
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
error
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
error
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
26

1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
error
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
error
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
error
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
error
2003
2004
2005
error
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036


3466
3467
3468
3469
3470
3471
3472
3473
3474
3475
3476
3477
3478
3479
3480
3481
3482
3483
3484
3485
3486
3487
3488
3489
3490
3491
3492
3493
3494
3495
3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
error
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
error
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
error
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
error
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3

In [ ]:
dataSet4Result = DataProcessing(dataSets[4])
print(len(dataSet4Result))
print('DONE')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
error
23
24
error
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
error
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
error
83
84
85
86
87
88
89
90
91
92
93
error
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273

1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
error
1905
1906
1907
1908
1909
1910
1911
1912
1913
error
1915
1916
1917
1918
1919
1920
1921
1922
1923
error
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
error
1939
1940
1941
1942
1943
1944
1945
1946
error
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046


3479
3480
3481
3482
3483
3484
3485
3486
3487
3488
3489
3490
3491
3492
3493
3494
3495
3496
3497
error
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
error
3676
3677
367

In [13]:
dataResult = dataSet0Result + dataSet1Result + dataSet2Result+ dataSet3Result+ dataSet4Result
print(len(dataResult))
dataResultDF = pd.DataFrame(dataResult)
dataResultDF.to_excel(excel_writer = "C:/Users/linhc/OneDrive/Documents/Sproj/DataCollection/Average14MinsDataId0.xlsx")

24314
